In [79]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
from tensorflow.examples.tutorials import mnist

In [3]:
mnist_data = mnist.input_data.read_data_sets("/tmp/data/", one_hot=False, reshape=False, )
train_data = mnist_data.train
valid_data = mnist_data.validation
test_data = mnist_data.test

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [65]:
def model_fn(features, labels, mode, params):
    x = features["x"]
    with tf.name_scope("conv1"):
        conv1 = tf.layers.conv2d(x, 32, [5, 5], padding="SAME", activation=tf.nn.elu)
        pool1 = tf.layers.max_pooling2d(conv1, [2, 2], 2)
    with tf.name_scope("conv2"):
        conv2 = tf.layers.conv2d(pool1, 64, [5, 5], padding="SAME", activation=tf.nn.elu)
        pool2 = tf.layers.max_pooling2d(conv2, [2, 2], 2)
    with tf.name_scope("fc1"):
        flat_pool2 = tf.reshape(pool2, [-1, 7*7*64])
        fc1 = tf.layers.dense(flat_pool2, 1024, activation=tf.nn.elu,
                              kernel_initializer=tf.variance_scaling_initializer(),
                              bias_initializer=tf.constant_initializer(0.))
        d1 = tf.layers.dropout(fc1, rate=params["dropout"],
                               training=(mode==tf.estimator.ModeKeys.TRAIN) )
    with tf.name_scope("output"):
        logits = tf.layers.dense(d1, 10)
        probs = tf.argmax(logits, axis=1)
        # doesn't work with eval_metric
#         accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logits, labels, k=1), tf.float32))
    
    predictions = {"logits": logits, "probs": probs}
    
    # need to return early for PREDICT MODE, other it will complain when continuing to construct
    # other parameters involving labels
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions)
    
    # for loss function, either works
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels))
#     loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    train_op = tf.train.AdamOptimizer(learning_rate=params["learning_rate"]).minimize(loss, global_step=tf.train.get_global_step())
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels, probs)} # in addition to loss
    
    return tf.estimator.EstimatorSpec(mode=mode,
                                      predictions=predictions, loss=loss,
                                      train_op=train_op, eval_metric_ops=eval_metric_ops)


params = {"learning_rate": 5e-4, "dropout": 0.5}

model = tf.estimator.Estimator(model_fn=model_fn, params=params, )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpimgy0gtt', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [66]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x':train_data.images},
                                                    y=train_data.labels.astype(np.int32), 
                                                    batch_size=256, shuffle=True,
                                                    num_epochs=None, num_threads=1)

model.train(input_fn=train_input_fn, steps=5000, )

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpimgy0gtt/model.ckpt.
INFO:tensorflow:loss = 2.30903, step = 1
INFO:tensorflow:global_step/sec: 34.6986
INFO:tensorflow:loss = 0.179387, step = 101 (2.883 sec)
INFO:tensorflow:global_step/sec: 36.1119
INFO:tensorflow:loss = 0.0972803, step = 201 (2.769 sec)
INFO:tensorflow:global_step/sec: 36.4844
INFO:tensorflow:loss = 0.0851764, step = 301 (2.741 sec)
INFO:tensorflow:global_step/sec: 36.3508
INFO:tensorflow:loss = 0.0228717, step = 401 (2.750 sec)
INFO:tensorflow:global_step/sec: 36.6088
INFO:tensorflow:loss = 0.0547687, step = 501 (2.732 sec)
INFO:tensorflow:global_step/sec: 36.1546
INFO:tensorflow:loss = 0.0403269, step = 601 (2.766 sec)
INFO:tensorflow:global_step/sec: 36.2808
INFO:tensorflow:loss = 0.0892885, step = 701 (2.756 sec)
INFO:tensorflow:global_step/sec: 36.2346
INFO:tensorflow:loss = 0.0532592, step = 801 (2.762 sec)
INFO:tensorflow:global_step/sec: 35.7317
INFO:tensorflow:

In [67]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x':train_data.images},
                                                    y=train_data.labels.astype(np.int32), 
                                                    batch_size=256, shuffle=True,
                                                    num_epochs=1, num_threads=1)
model.evaluate(train_input_fn)

INFO:tensorflow:Starting evaluation at 2017-09-22-07:42:39
INFO:tensorflow:Restoring parameters from /tmp/tmpimgy0gtt/model.ckpt-5000
INFO:tensorflow:Finished evaluation at 2017-09-22-07:42:41
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.996127, global_step = 5000, loss = 0.0110076


{'accuracy': 0.99612725, 'global_step': 5000, 'loss': 0.011007594}

In [68]:
valid_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': valid_data.images},
    y=valid_data.labels.astype(np.int32),
    batch_size=128,
    shuffle=False,
    num_epochs=1
)

model.evaluate(valid_input_fn)

INFO:tensorflow:Starting evaluation at 2017-09-22-07:42:42
INFO:tensorflow:Restoring parameters from /tmp/tmpimgy0gtt/model.ckpt-5000
INFO:tensorflow:Finished evaluation at 2017-09-22-07:42:42
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.9888, global_step = 5000, loss = 0.0630444


{'accuracy': 0.98879999, 'global_step': 5000, 'loss': 0.063044399}

In [83]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': test_data.images},
#     y=test_data.labels.astype(np.int32),
    batch_size=128,
    shuffle=False,
    num_epochs=1
)
test_yhat = pd.DataFrame(list(model.predict(test_input_fn))) # evaluate now

INFO:tensorflow:Restoring parameters from /tmp/tmpimgy0gtt/model.ckpt-5000


In [84]:
test_yhat.head()

,logits,probs
0,"[-8.57827, 2.45554, 0.988721, -0.621002, 2.907...",7
1,"[12.8002, 14.2683, 50.5243, -12.7172, -13.0333...",2
2,"[-8.64605, 29.8617, -6.6989, -12.2865, 12.2244...",1
3,"[32.6897, -2.60047, 8.46332, -16.8167, -9.3153...",0
4,"[-7.00146, -0.305858, -7.15816, -14.0377, 36.5...",4


In [85]:
np.mean(test_yhat.probs == test_data.labels)

0.98780000000000001